# Testing YAML I/O with units

In [1]:
import astropy.units as u
import numpy as np
import yaml
import asdf

from hydrad_tools.configure import Configure

In [29]:
base_config = {
    'general': {
        'total_time': 5e4 * u.s,
        'loop_length': 80 * u.Mm,
        'footpoint_height': 5e8 * u.cm,
        'output_interval': 5*u.s,
        'loop_inclination': 0*u.deg,
        'logging_frequency': 1000,
        'write_file_physical': True,
        'write_file_ion_populations': False,
        'write_file_hydrogen_level_populations': False,
        'write_file_timescales': False,
        'write_file_equation_terms': False,
        'heat_flux_limiting_coefficient': 1./6.,
        'heat_flux_timestep_limit': 1e-10*u.s,
        'use_kinetic_model': False,
        'minimum_collisional_coupling_timescale': 0.01*u.s,
        'force_single_fluid': False,
        #'tabulated_gravity_profile':,
        #'tabulated_cross_section_profile':,
    },
    'initial_conditions': {
        'footpoint_temperature': 2e4 * u.K,
        'footpoint_density': 1e11 * u.cm**(-3),
        'isothermal': False,
        'heating_location': 40 * u.Mm,
        'heating_scale_height': 1e300*u.cm,
        'heating_range_lower_bound': 1e-8*u.erg/u.s/(u.cm**3),
        'heating_range_upper_bound': 1e2*u.erg/u.s/(u.cm**3),
        'heating_range_step_size': 0.01,
        'heating_range_fine_tuning': 10000.0,
        'use_tabulated_gravity': False,
    },
    'radiation': {
        'use_power_law_radiative_losses': True,
        'decouple_ionization_state_solver': False,
        'density_dependent_rates': False,
        'optically_thick_radiation': False,
        'nlte_chromosphere': False,
        'ranges_dataset': 'ranges',
        'emissivity_dataset': 'chianti_v7',
        'abundance_dataset': 'asplund',
        'rates_dataset': 'chianti_v7',
        'elements_equilibrium': ['Fe'],
        'elements_nonequilibrium': [],
    },
    'heating': {
        'heat_electrons': True,
        'background_heating': False,
        'beam_heating': False,
        'alfven_wave_heating': False,
        'events': [
            {'time_start': 0.*u.s, 
             'rise_duration': 100*u.s,
             'decay_duration': 100*u.s, 
             'total_duration': 200*u.s,
             'location': 4e9*u.cm, 
             'scale_height': 1e300 * u.cm,
             'rate': 0.1 *u.erg/u.s/(u.cm**3),},
        ],
    },
    'solver': {
        'epsilon': 0.01,
        'safety_radiation': 0.1,
        'safety_conduction': 1.0,
        'safety_advection': 1.0,
        'safety_atomic': 1.0,
        'safety_viscosity': 1.0,
        'cutoff_ion_fraction':1e-6,
        'epsilon_d':0.1,
        'epsilon_r':1.8649415311920072,
        'timestep_increase_limit': 0.05,
        'relative_viscous_timescale': None,
        'minimum_radiation_temperature': 2e4*u.K,
        'zero_over_temperature_interval': 5.0e2*u.K,
        'minimum_temperature': 1e4*u.K,
        'maximum_optically_thin_density': 1e12*u.cm**(-3),
    },
    'grid': {
        'adapt': True,
        'adapt_every_n_time_steps': 10,
        'minimum_cells': 150,
        'maximum_cells': 30000,
        'maximum_refinement_level': 12,
        'minimum_delta_s': 1.0*u.cm,
        'maximum_variation': 0.10,
        'refine_on_density': True,
        'refine_on_electron_energy': True,
        'refine_on_hydrogen_energy': False,
        'minimum_fractional_difference': 0.10,
        'maximum_fractional_difference': 0.20,
        'linear_restriction': True,
        'enforce_conservation': False,
    }
} 

In [30]:
af = asdf.AsdfFile(base_config)

In [31]:
af.write_to('big_example.asdf')

In [19]:
c = Configure(base_config)

In [20]:
c.save_config(c.config, 'base_config.asdf')

In [28]:
bc2 = Configure.load_config('big_example.asdf')

ValueError: Input object does not appear to be ASDF file or FITS with ASDF extension

In [22]:
c2 = Configure(bc2)

In [23]:
print(c2.heating_cfg)

0.0 0.0 0.0

1

4000000000.0 1e+300 0.1 0.0 100.0 100.0 200.0

Configuration file generated by hydrad_tools on 2018-06-07_15.13.32


In [3]:
test_dict = {
    'general': {
        'loop_length': 100*u.cm,
        'total_time': 5e3 * u.s,
        'logging_frequency': 1000,
    },
    'initial_conditions': {
        'footpoint_temperature': 1e4*u.K,
        'use_gravity_file': False,
    }
}
test_dict2 = {'general': {'loop_length': 100,
                          'total_time': 5e3,
                          'logging_frequency': 1000,},
              'initial_conditions': {'footpoint_temperature': 1e4,
                                     'use_gravity_file': False,}}

In [4]:
af = asdf.AsdfFile(test_dict)

In [5]:
af.write_to('test.asdf')

In [6]:
with asdf.open('test.asdf') as f:
    test_dict_rt = f.tree

In [8]:
f.comments

[b'ASDF_STANDARD 1.2.0']

In [6]:
with open('test.yml','w') as f:
    test_dict_io = {}
    for k in test_dict:
        test_dict_io[k] = {}
        for v in test_dict[k]:
            if isinstance(test_dict[k][v], u.Quantity):
                test_dict_io[k][v] = {'value': test_dict[k][v].value, 'unit': test_dict[k][v].unit.to_string()}
            else:
                test_dict_io[k][v] = test_dict[k][v]
    yaml.dump(test_dict_io, f)

In [14]:
with open('test.yml','r') as f:
    foo = yaml.load(f)
    new_foo = {}
    for k in foo:
        new_foo[k] = {}
        for v in foo[k]:
            if type(foo[k][v]) is dict and 'unit' in foo[k][v] and 'value' in foo[k][v]:
                new_foo[k][v] = u.Quantity(foo[k][v]['value'],foo[k][v]['unit'])
            else:
                new_foo[k][v] = foo[k][v]

In [16]:
test_dict

{'general': {'logging_frequency': 1000,
  'loop_length': <Quantity 100. cm>,
  'total_time': <Quantity 5000. s>},
 'initial_conditions': {'footpoint_temperature': <Quantity 10000. K>,
  'use_gravity_file': False}}

In [17]:
new_foo == test_dict

True

In [33]:
default_config = {
    'general': {
        'output_interval': 1*u.s,
        'logging_frequency': 1000,
        'write_file_physical': True,
        'write_file_ion_populations': False,
        'write_file_hydrogen_level_populations': False,
        'write_file_timescales': False,
        'write_file_equation_terms': False,
        'heat_flux_timestep_limit': 1e-10*u.s,
        'use_kinetic_model': False,
        'minimum_collisional_coupling_timescale': 0.01*u.s,
        'force_single_fluid': False,
        #'tabulated_gravity_profile':,
        #'tabulated_cross_section_profile':,
    },
    'initial_conditions': {
        'isothermal': False,
        'heating_range_lower_bound': 1e-8*u.erg/u.s/(u.cm**3),
        'heating_range_upper_bound': 1e2*u.erg/u.s/(u.cm**3),
        'heating_range_step_size': 0.01,
        'heating_range_fine_tuning': 10000.0,
        'use_tabulated_gravity': False,
    },
    'radiation': {
        'use_power_law_radiative_losses': True,
        'decouple_ionization_state_solver': False,
        'density_dependent_rates': False,
        'optically_thick_radiation': False,
        'nlte_chromosphere': False,
        'ranges_dataset': 'ranges',
        'emissivity_dataset': 'chianti_v7',
        'abundance_dataset': 'asplund',
        'rates_dataset': 'chianti_v7',
        'elements_equilibrium': ['Fe'],
        'elements_nonequilibrium': [],
    },
    'heating': {
        'heat_electrons': True,
        'background_heating': False,
        'beam_heating': False,
        'alfven_wave_heating': False,
        'events': [],
    },
    'solver': {
        'epsilon': 0.01,
        'safety_radiation': 0.1,
        'safety_conduction': 1.0,
        'safety_advection': 1.0,
        'safety_atomic': 1.0,
        'safety_viscosity': 1.0,
        'cutoff_ion_fraction':1e-6,
        'epsilon_d':0.1,
        'epsilon_r':1.8649415311920072,
        'timestep_increase_limit': 0.05,
        'relative_viscous_timescale': None,
        'minimum_radiation_temperature': 2e4*u.K,
        'zero_over_temperature_interval': 5.0e2*u.K,
        'minimum_temperature': 1e4*u.K,
        'maximum_optically_thin_density': 1e12*u.cm**(-3),
    },
    'grid': {
        'adapt': True,
        'adapt_every_n_time_steps': 10,
        'minimum_cells': 150,
        'maximum_cells': 30000,
        'maximum_refinement_level': 12,
        'minimum_delta_s': 1.0*u.cm,
        'maximum_variation': 0.10,
        'refine_on_density': True,
        'refine_on_electron_energy': True,
        'refine_on_hydrogen_energy': False,
        'minimum_fractional_difference': 0.10,
        'maximum_fractional_difference': 0.20,
        'linear_restriction': False,
        'enforce_conservation': True,
    }
} 

In [34]:
Configure.save_config(default_config,'/Users/willbarnes/Documents/work/codes/hydrad_tools/defaults.asdf')